<a href="https://colab.research.google.com/github/harvard-ml-courses/a-cs281-demo/blob/master/18_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -qU plotly torch daft opt-einsum networkx
!rm -fr start; git clone --single-branch -b demos2018 -q https://github.com/harvard-ml-courses/cs281-demos start; cp -f start/cs281.py cs281.py


tcmalloc: large alloc 1073750016 bytes == 0x59056000 @  0x7f99e74ab2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [0]:
import torch
import torch.distributions as ds
import daft

In [0]:
# LDS Parameters
Sigma = torch.eye(10).unsqueeze(0)
Sigma2 = torch.eye(20).unsqueeze(0)
A = torch.rand(10, 10)
C = torch.rand(20, 10)

In [0]:
# Sample through LDS
x = []
z = [ds.MultivariateNormal(torch.rand(1, 10), Sigma).sample(torch.Size([5])).squeeze(1)]
for t in range(10):
    mu = torch.einsum("xz,bz->bx", [C, z[-1]])
    x.append(ds.MultivariateNormal(mu, Sigma2).sample())
    mu2 = torch.einsum("xz,bz->bx", [A, z[-1]])
    z.append(ds.MultivariateNormal(mu2, Sigma).sample())

In [0]:
# Non-Linear LDS Parameters
Sigma = torch.eye(10).unsqueeze(0)
Sigma2 = torch.eye(20).unsqueeze(0)
A = torch.rand(10, 10)
C = torch.rand(30, 10)
D = torch.rand(20, 30)

In [0]:
# Sample through non-linear DS
z = [ds.MultivariateNormal(torch.rand(1, 10), Sigma).sample(torch.Size([5])).squeeze(1)]
for t in range(10):
    h = torch.sigmoid(torch.einsum("xz,bz->bx", [C, z[-1]]))
    mu = torch.sigmoid(torch.einsum("bx,yx->by", [h, D]))
    x.append(ds.MultivariateNormal(mu, Sigma2).sample())
    mu2 = torch.einsum("xz,bz->bx", [A, z[-1]])
    z.append(ds.MultivariateNormal(mu2, Sigma).sample())